# Spatial Data Processing with libpysal,  Pandas and Geopandas


In [ ]:
# By convention, we use these shorter names
import libpysal as ps
import pandas as pd
import numpy as np
import geopandas as gpd

## libpysal 

libpysal has a command that it uses to get the paths of its example datasets. Let's work with a commonly-used dataset first. 

In [ ]:
ps.examples.available()

In [ ]:
ps.examples.explain('us_income')

In [ ]:
csv_path = ps.examples.get_path('usjoin.csv')
csv_path

In [ ]:
f = ps.io.open(csv_path)
f.header[0:10]

In [ ]:
y2009 = f.by_col('2009')

In [ ]:
y2009[0:10]

### Working with shapefiles

We can also work with local files outside the built-in examples.

To read in a shapefile, we will need the path to the file.

In [ ]:
shp_path = 'data/texas.shp'
print(shp_path)

Then, we open the file using the `ps.io.open` command:

In [ ]:
f = ps.io.open(shp_path)

`f` is what we call a "file handle." That means that it only *points* to the data and provides ways to work with it. By itself, it does not read the whole dataset into memory. To see basic information about the file, we can use a few different methods. 

For instance, the header of the file, which contains most of the metadata about the file:

In [ ]:
f.header

To actually read in the shapes from memory, you can use the following commands:

In [ ]:
f.by_row(14) # gets the 14th shape from the file

In [ ]:
all_polygons = f.read() # reads in all polygons from memory

In [ ]:
len(all_polygons)

So, all 254 polygons have been read in from file. These are stored in libpysal shape objects, which can be used by libpysal and can be converted to other Python shape objects.

They typically have a few methods. So, since we've read in polygonal data, we can get some properties about the polygons. Let's just have a look at the first polygon:

In [ ]:
all_polygons[0:5]

In [ ]:
all_polygons[0].centroid #the centroid of the first polygon

In [ ]:
all_polygons[0].area

In [ ]:
all_polygons[0].perimeter

While in the Jupyter Notebook, you can examine what properties an object has by using the tab key.

In [ ]:
polygon = all_polygons[0]

In [ ]:
polygon. #press tab when the cursor is right after the dot

### Working with Data Tables

In [ ]:
dbf_path = "data/texas.dbf"
print(dbf_path)

When you're working with tables of data, like a `csv` or `dbf`, you can extract your data in the following way. Let's open the dbf file we got the path for above.

In [ ]:
f = ps.io.open(dbf_path)

Just like with the shapefile, we can examine the header of the dbf file.

In [ ]:
f.header

So, the header is a list containing the names of all of the fields we can read.
If we just wanted to grab the data of interest, `HR90`, we can use either `by_col` or `by_col_array`, depending on the format we want the resulting data in:

In [ ]:
HR90 = f.by_col('HR90')
print(type(HR90).__name__, HR90[0:5])
HR90 = f.by_col_array('HR90')
print(type(HR90).__name__, HR90[0:5])

As you can see, the `by_col` function returns a list of data, with no shape. It can only return one column at a time:

In [ ]:
HRs = f.by_col('HR90', 'HR80')

This error message is called a "traceback," as you see in the top right, and it usually provides feedback on why the previous command did not execute correctly. Here, you see that one-too-many arguments was provided to `__call__`, which tells us we cannot pass as many arguments as we did to `by_col`.

If you want to read in many columns at once and store them to an array, use `by_col_array`:

In [ ]:
HRs = f.by_col_array('HR90', 'HR80')

In [ ]:
HRs[0:10]

It is best to use `by_col_array` on data of a single type. That is, if you read in a lot of columns, some of them numbers and some of them strings, all columns will get converted to the same datatype:

In [ ]:
allcolumns = f.by_col_array(['NAME', 'STATE_NAME', 'HR90', 'HR80'])

In [ ]:
allcolumns

Note that the numerical columns, `HR90` & `HR80` are now considered strings, since they show up with the single tickmarks around them, like `'0.0'`.

These methods work similarly for `.csv` files as well.

## Geopandas & pandas

In [ ]:
shp_path = ps.examples.get_path('NAT.shp')
df = gpd.read_file(shp_path)

This reads in *the entire database table* and adds a column to the end, called `geometry`, that stores the geometries read in from the shapefile. 

In [ ]:
df.head()

The `read_file` function only works on shapefile/dbf pairs. If you need to read in data using CSVs, use pandas directly:

In [ ]:
df.plot()

In [ ]:
df.groupby("STATE_NAME").size()

In [ ]:
df.query('STATE_NAME == "California"')

In [ ]:
df.STATE_NAME == 'California'

## Create a new dataframe by subsetting for California

In [ ]:
california = df[df.STATE_NAME == 'California']

In [ ]:
california.head()

In [ ]:
california.plot()

In [ ]:
california.shape 

## More selection

In [ ]:
target = ['California', 'Arizona']

In [ ]:
df['STATE_NAME'].isin(target).head()

In [ ]:
ca_az = df[df['STATE_NAME'].isin(target)]

In [ ]:
ca_az.plot()

In [ ]:
target = ['California', 'Arizona', 'Nevada']

In [ ]:
ca_az_nv = df[df['STATE_NAME'].isin(target)]

In [ ]:
ca_az_nv.plot()

In [ ]:
ca_az_nv.plot(column='STATE_NAME', categorical=True)

In [ ]:
ca_az_nv.shape

## Get Polygon Centroids

In [ ]:
gdf = ca_az_nv
lon = gdf.centroid.map(lambda p: p.x).values
lat = gdf.centroid.map(lambda p: p.y).values


In [ ]:
%pylab inline


In [ ]:
plot(lon, lat, '.')

In [ ]:
from shapely.geometry import Point

In [ ]:
points = [Point(x,y) for x,y in zip(lon,lat)]

In [ ]:
centroids = gpd.GeoDataFrame(geometry=points)

## Plotting with Layers

In [ ]:
base = gdf.plot(color='white', edgecolor='black')
centroids.plot(ax=base, marker='o', color='red', markersize=2)